In [2]:
import pandas
import looker_sdk # if unavailable, pip install looker_sdk

# Important note about Authorization

The following line is authorizing you to use the API on our instance, but it requires you to have a looker.ini file with the following:

```
[Looker]
base_url = "https://[COMPANY_NAME].looker.com:19999"
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
```

If you don't have a `client_id` and `client_secret`, you will need
to get your API3 keys from your Looker Admin. If you are a Looker Admin,
head to Admin > Users and edit a user. You'll be able to edit / create API3
keys from this menu. 

**Keep your API3 Secret SAFE and do not put any code on GitHub that has your client secret exposed.**

In [ ]:
# Read the text above for more info on how to initiate your API session
admin_sdk = looker_sdk.init31()

# Using a csv to see users' current teams and roles

In [ ]:
# Removes all users in a list of user_ids from a single group

def remove_users_from_group(admin_sdk, user_id_list, group_id):
    # Iterate through user_id_list
    for user_id in user_id_list:
      # Delete the user from the group
      admin_sdk.delete_group_user(group_id, user_id)

In [ ]:
# Adds all users in a list of user_ids to a single group

def add_users_to_group(admin_sdk, user_id_list, group_id):
    # Iterate through user_id_list
    for user_id in user_id_list:
        # Generate the body to pass through the API call
        body = {
            'user_id': user_id
        }
        # Add the user to the group
        admin_sdk.add_group_user(group_id, body)

# Create new groups from a base name
This function pairs well with the new role function below

In [24]:
def create_new_groups(admin_sdk, group_base_name, nest_groups:bool):
    '''
    Creates new Viewer, Explorer, and Developer groups with a base name prefix.
    
        nest_groups -   Nests Developer inside Explorer, and Explorer inside Viewer.
                        This ensures that Developers will inherit Explorer and Viewer permissions
                        and Explorers will inherit viewer permissions.
        
        returns a list of the new names to be passed into create_new_role function
    '''
    new_group_names = []
    group_suffixes = ['Viewer', 'Explorer', 'Developer']
    for group_suffix in group_suffixes:
        full_group_name = f'{group_base_name} {group_suffix}'
        new_group_names.append(full_group_name)
        print(new_group_names)
        body = {
            'name': full_group_name
        }
        new_group = admin_sdk.create_group(body)

        if nest_groups:

            # Store the value of the viewer group for nesting later
            if group_suffix == "Viewer":
                viewer_group_id = new_group.id

            # All explorers should be viewers, so nest explorer group
            # inside of viewer group
            elif group_suffix == "Explorer":
                explorer_group_id = new_group.id
                explorer_body = {
                    "group_id": explorer_group_id
                }
                admin_sdk.add_group_group(viewer_group_id, explorer_body)
            
            # All Developers should be explorers and viewers, so nest
            # Inside of explorer group, which will be nested inside viewer group
            else:
                developer_group_id = new_group.id
                developer_body = {
                    "group_id": developer_group_id
                }
                admin_sdk.add_group_group(explorer_group_id, developer_body)
    return new_group_names
        

In [ ]:
def create_new_role(admin_sdk, new_role_name, new_role_groups: list):
    '''
    Make sure groups are defined first! This will auto 
    assign groups for you if they exist already

    new_role_groups: list - should be a list of names of 
    groups to add to the new role
    '''

    # This requires an existing permission set and model set
    # You can define a new permission set and model set by using the
    # permission_set: {} and model_set: {} params
    # See the API Explorer in your Looker instance for more info
    body = {
        "name": new_role_name
        "permission_set_id": base_permission_set_id,
        "model_set_id": base_model_set_id,
    }

    # Create the new role
    new_role = admin_sdk.create_role(body)

    # Get the id of the role we just created
    new_role_id = new_role.id

    # Get the group ids currently associated with the role
    current_role_group_ids = [group.id for group in admin_sdk.role_groups(role_id)]

    # Get the group ids for the new role groups
    new_role_group_ids = [new_role_group.id for new_role_group in admin_sdk.all_groups() if new_role_group.name in new_role_groups]

    # Add the new_role_group to the current_role_group_ids
    updated_role_group_ids = current_role_group_ids + new_role_group_ids

    admin_sdk.set_role_groups(role_id, updated_role_group_ids)

## CODE EXAMPLES: Getting user info from a csv and using it in the API

In [ ]:
# Store csv contents in pandas dataframe
# We have a "Team" and "Display_Name" field in this csv

csv_path = '..\looker_users.csv' # Update this with your csv path
df = pd.read_csv(csv_path)

In [ ]:
# Storing team value for searching
team = 'Marketing'

# Store people from a specific team
user_list = df[df['Team'] == team].sort_values('Display_Name')

# Get a list of user ids to pass through functions
user_id_list = list(user_list['id'])

In [ ]:
# Obtaining group ids via a search with wildcards
group_id = admin_sdk.search_groups(name='Marketing')[0].id

In [ ]:
# Adding users to group with the id above. You can hard code in the ID
# If you find it via the Admin > Groups page in Looker

# Line left commented because we don't want to accidentally add users
# add_users_to_group(admin_sdk, user_id_list, group_id)

## Creating full group names to pass through the create_new_role() function

In [ ]:
group_base_name = 'Marketing'
nest_groups = True

'''
The line below will create new groups and connect them to new roles
with basic model set and permission sets. Uncomment to make a live 
API call and add the new groups and roles
'''
# The role name and group name are the same, in our case
# new_group_list = create_new_groups(admin_sdk, group_base_name, nest_groups)
# for group_name in new_group_list:
#   create_new_role(admin_sdk, group_name, group_name)